In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')
def setup_seed(seed=0):
    import torch
    import os
    import numpy as np
    import random
    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    if torch.cuda.is_available():
        # torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
        torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
import nltk
import re
from nltk.corpus import stopwords
import numpy as np
nltk.download('stopwords')
eng_stopwords = stopwords.words('english')
def clean_text(text):
    #text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    words = [w for w in words if w not in eng_stopwords]
    return ' '.join(words)
def sliding_windows(data,prediction, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = prediction[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)
from sklearn.metrics import mean_squared_error
def process_data(data,periods,year):
  data['clean_review'] = data.text.apply(clean_text)
  data
  if year==30:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_30_US.csv',index_col=[0])
  elif year==10:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_10_US.csv',index_col=[0])
  elif year==5:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_5_US.csv',index_col=[0])
  elif year==0.3:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_3m_US.csv',index_col=[0])
  interest_rate=interest_rate.sort_values(by='Date')
  interest_rate['range']=list(range(len(interest_rate)))
  data['change']=np.nan
  for i in range(len(data)):
    try:
      index=int(interest_rate[interest_rate['Date']<=int(data.iloc[i,0])]['range'].tail(1))
      data.iloc[i,-1]=(interest_rate.iloc[index+periods,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
    except:
      pass
  new_raw=data.dropna()
  
  return new_raw
def split_data(data,ratio,features,seq_length):
  vectorizer_tfidf=TfidfVectorizer(max_features=features)
  total_vsm_tfidf=vectorizer_tfidf.fit_transform(data.clean_review).toarray()
  seq_length = 4
  x, y = sliding_windows(total_vsm_tfidf,list(data.change), seq_length)
  train_size = int(len(x) *ratio)
  test_size = len(x) - train_size
  train_Y=np.array(x[0:train_size])
  from sklearn.preprocessing import RobustScaler
  train_Y=np.array(y[0:train_size])
  setup_seed(1)
  scaler=RobustScaler().fit(train_Y.reshape(-1, 1) )
  train_Y = scaler.transform(train_Y.reshape(-1, 1)).reshape(1,-1)[0]
  train_Y = Variable(torch.Tensor(train_Y))
  train_X = Variable(torch.Tensor(np.array(x[0:train_size])))
  test_X = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
  test_Y = np.array(y[train_size:len(y)])
  return train_X,train_Y,test_X,test_Y
def taining(train_X,train_Y,test_X,test_Y,data,ratio,seq_length,features,l2,seed):
  #train_X,train_Y,test_X,test_Y=split_data(data,ratio,features,seq_length)
  num_epochs = 400
  learning_rate = 0.01
  input_size = features
  hidden_size = 2
  num_layers = 1
  num_classes = 1
  setup_seed(seed)
  lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
  criterion = torch.nn.MSELoss()    # mean-squared error for regression
  optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=l2)
  from sklearn.metrics import mean_squared_error
  for epoch in range(num_epochs):
      outputs = lstm(train_X)
      optimizer.zero_grad()
      loss = criterion(outputs, train_Y[0])
      loss.backward()
      optimizer.step()
      predTrain = lstm(test_X)
      print(predTrain.shape)
      tmp=pd.DataFrame(list(test_Y))
      tmp["1"]=list(predTrain.detach().numpy().reshape(1,-1))[0]
      r = mean_squared_error(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])
      print('IC',round(tmp.corr().iloc[0,1],5),
            'IC spear',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[0],5),\
            'P_value',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[1],5),
            'loss',round(r,5),
            'variance',round(np.array(test_Y).var(),5))
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import os
path = "/content/gdrive/MyDrive/JPM/centralbank_analysis/data/FOMC/statement/"
os.chdir(path)
os.listdir(path)
file_list=[i for i in os.listdir(path) if 'txt' in i]

In [4]:
file_list_time=[file_list[i].split('-')[0][-4:]+file_list[i].split('-')[1]+file_list[i].split('-')[2][:2] for i in range(len(file_list))]

In [5]:
repeat_sentence='The Federal Reserve, the central bank of the United States, provides the nation with a safe, flexible, and stable monetary and financial system. Monetary Policy Principles and Practice Review of Monetary Policy Strategy, Tools, and Communications Banking Applications & Legal Developments Reserve Bank Payment Services & Data Financial Market Utilities & Infrastructures Exchange Rates and International Data Micro Data Reference Manual (MDRM) Money Stock and Reserve Balances' 

In [6]:
statement_text=dict()
for i in range(len(file_list)):
  f=open(path+file_list[i],'r',encoding='latin1')
  f1=f.readlines()
  contents=[]
  for line in f1:
    if ('[SECTION]' not in line):
      if len(line.strip().split(' '))>=5:
        if ('Voting for the FOMC monetary policy action were' in line):
          break
        contents.append(line.strip())
  text = ' '.join(contents)
  text=text.replace('Â\xa0',' ')
  text=text.replace(repeat_sentence,"")
  if len(text.split(' '))>100:
    statement_text[file_list_time[i]]='statement '+text
  else:
    print(i,len(text.split(' ')),text)

3 99 Chairman Alan Greenspan announced today that the Federal Open Market Committee decided to increase slightly the degree of pressure on reserve positions. The action is expected to be associated with a small increase in short-term money market interest rates. The decision was taken to move toward a less accommodative stance in monetary policy in order to sustain and enhance the economic expansion. Chairman Greenspan decided to announce this action immediately so as to avoid any misunderstanding of the Committee's purposes, given the fact that this is the first firming of reserve market conditions by the Committee since early 1989.
15 36 Chairman Alan Greenspan announced today that the Federal Reserve will increase slightly the degree of pressure on reserve positions. This action is expected to be associated with a small increase in short-term money market interest rates.
41 71 Chairman Alan Greenspan announced today that the Federal Open Market Committee decided to decrease slightly

In [7]:
path = "/content/gdrive/MyDrive/JPM/centralbank_analysis/data/FOMC/speech/"
os.chdir(path)
os.listdir(path)
file_list=[i for i in os.listdir(path) if 'txt' in i]

In [8]:
file_list_time_speech=[file_list[i].split('-')[0][-4:]+file_list[i].split('-')[1]+file_list[i].split('-')[2][:2] for i in range(len(file_list))]

In [9]:
speech_dicter={}
path_save = "/content/gdrive/MyDrive/JPM/centralbank_analysis/data/FOMC/text_summarization/speech/"
for i in range(len(file_list)):
  #print(i,file_list[i])
  f=open(path+file_list[i],'r',encoding='latin1')
  f1=f.readlines()
  contents=[]
  for line in f1:
    #print(line)
    if ('[SECTION]' not in line):
      if len(line.strip().split(' '))>=5:
        contents.append(line.strip().replace('Â',' '))
  
  text = ' '.join(contents)
  text=text.replace('Â\xa0',' ')
  text=text.replace(repeat_sentence,"")
  #print(text)
  if (len(text.split(' '))<100):
    print('!',i,text)
  else:
    speech_dicter[file_list_time_speech[i]]='speech '+text

! 84 Reflections on the Capital Goods Overhang
! 104 
! 110 Corporate governance, the subject of our conference, has evolved over the past century to more effectively promote the allocation of the nation s savings to its most productive uses.  And, generally speaking, the resulting structure of business incentives, reporting, and accountability has served us well.  We could not have achieved our current level of national productivity if corporate governance had been deeply flawed.
! 154 
! 159 
! 169 
! 188 
! 205 
! 254 
! 321 Asset Prices and Monetary Policy
! 333 Thank you, Tom.  On behalf of the Federal Reserve, I am pleased to be with you all as we unveil the newly designed $20 note.
! 370 I am pleased to appear once again before the Independent Community Bankers of America.  There can be little doubt that the business environment in which you have operated over the past year has been challenging.
! 406 
! 469 
! 519 Thank you. I am pleased to have the opportunity to help launch 

In [10]:
import pandas as pd
raw_data_speech=pd.DataFrame({'date':speech_dicter.keys(),'text':speech_dicter.values(),'type':['speech' for i in range(len(speech_dicter.values()))]})
raw_data_speech.sort_values(by='date',inplace=True)
raw_data_speech

,date,text,type
1083,19960613,speech Remarks by Chairman Alan Greenspan Bank...,speech
488,19960618,"speech Remarks by Governor Edward W. Kelley, J...",speech
15,19960908,speech Monetary Policy Objectives and Strategy...,speech
1232,19960919,speech Remarks by Chairman Alan Greenspan Regu...,speech
1236,19961002,speech Remarks by Governor Lawrence B. Lindsey...,speech
...,...,...,...
810,20220221,speech At the American Bankers Association Co...,speech
700,20220224,"speech At the Economic Forecast Project, Univ...",speech
737,20220318,speech At Fed Listens: Helping Youth ThriveÃ¢...,speech
805,20220321,speech Chair Pro Tempore Jerome H. Powell At ...,speech


In [11]:
raw_data_statement=pd.DataFrame({'date':statement_text.keys(),'text':statement_text.values(),'type':['statement' for i in range(len(statement_text.values()))]})
raw_data_statement.sort_values(by='date',inplace=True)
raw_data_statement

,date,text,type
186,19940517,statement The Federal Reserve today announced ...,statement
146,19940816,statement The Federal Reserve announced today ...,statement
177,19941115,statement The Federal Reserve Board today appr...,statement
153,19950201,statement The Federal Reserve Board today appr...,statement
25,19960131,statement The Federal Reserve today announced ...,statement
...,...,...,...
149,20210922,statement For release at 2:00 p.m. EDT The Fe...,statement
88,20211103,statement For release at 2:00 p.m. EDT The Fe...,statement
33,20211215,statement For release at 2:00 p.m. EST The Fe...,statement
66,20220126,statement For release at 2:00 p.m. EST Indica...,statement


In [12]:
raw_data=pd.concat([raw_data_statement,raw_data_speech],axis=0)
raw_data.sort_values(by='date',inplace=True)
eng_stopwords = stopwords.words('english')
def clean_text(text):
    #text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    words = [w for w in words if w not in eng_stopwords]
    return ' '.join(words)

raw_data['clean_review'] = raw_data.text.apply(clean_text)
raw_data

,date,text,type,clean_review
186,19940517,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced two ...
146,19940816,statement The Federal Reserve announced today ...,statement,statement federal reserve announced today foll...
177,19941115,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...
153,19950201,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...
25,19960131,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced foll...
...,...,...,...,...
700,20220224,"speech At the Economic Forecast Project, Univ...",speech,speech economic forecast project university ca...
188,20220316,statement For release at 2:00 p.m. EDT Indica...,statement,statement release p edt indicators economic ac...
737,20220318,speech At Fed Listens: Helping Youth ThriveÃ¢...,speech,speech fed listens helping youth thrive discus...
805,20220321,speech Chair Pro Tempore Jerome H. Powell At ...,speech,speech chair pro tempore jerome h powell polic...


In [20]:
features=150
seq_length = 5
ratio_=0.7
periods=5
interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_30_US.csv',index_col=[0])
interest_rate=interest_rate.sort_values(by='Date')
interest_rate['range']=list(range(len(interest_rate)))
raw_data['change']=np.nan
for i in range(len(raw_data)):
  try:
    index=int(interest_rate[interest_rate['Date']<=int(raw_data.iloc[i,0])]['range'].tail(1))
    raw_data.iloc[i,-1]=(interest_rate.iloc[index+periods,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
  except:
    pass
new_raw=raw_data.dropna()
vectorizer_freq = CountVectorizer(max_features = features) 
total_vsm_freq = vectorizer_freq.fit_transform(new_raw.clean_review).toarray()
print("以词频为元素的文本-单词矩阵的维度是：\n\n",total_vsm_freq.shape)
from scipy import stats
vectorizer_tfidf=TfidfVectorizer(max_features=features)
total_vsm_tfidf=vectorizer_tfidf.fit_transform(new_raw.clean_review).toarray()
x, y = sliding_windows(total_vsm_tfidf,list(new_raw.change), seq_length)
train_size = int(len(x) * ratio_)
test_size = len(x) - train_size
train_Y=np.array(x[0:train_size])
from sklearn.preprocessing import RobustScaler
train_Y=np.array(y[0:train_size])
scaler =RobustScaler().fit(train_Y.reshape(-1, 1) )
train_Y = scaler.transform(train_Y.reshape(-1, 1)).reshape(1,-1)[0]
train_Y=Variable(torch.Tensor(train_Y))
train_X = Variable(torch.Tensor(np.array(x[0:train_size])))
test_X = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
test_Y = np.array(y[train_size:len(y)])

以词频为元素的文本-单词矩阵的维度是：

 (1512, 150)


In [22]:
taining(train_X,train_Y,test_X,test_Y,new_raw,0.7,5,150,0.001,4)


torch.Size([452, 1])
IC -0.00904 IC spear -0.03903 P_value 0.40779 loss 0.04776 variance 0.00076
Epoch: 0, loss: 0.01447
torch.Size([452, 1])
IC -0.0134 IC spear -0.04644 P_value 0.32453 loss 0.04082 variance 0.00076
Epoch: 1, loss: 0.01060
torch.Size([452, 1])
IC -0.01826 IC spear -0.05425 P_value 0.24973 loss 0.03448 variance 0.00076
Epoch: 2, loss: 0.00736
torch.Size([452, 1])
IC -0.02337 IC spear -0.06156 P_value 0.1914 loss 0.02879 variance 0.00076
Epoch: 3, loss: 0.00475
torch.Size([452, 1])
IC -0.02845 IC spear -0.06604 P_value 0.16101 loss 0.02378 variance 0.00076
Epoch: 4, loss: 0.00278
torch.Size([452, 1])
IC -0.03327 IC spear -0.06984 P_value 0.13821 loss 0.01946 variance 0.00076
Epoch: 5, loss: 0.00139
torch.Size([452, 1])
IC -0.03762 IC spear -0.07433 P_value 0.11454 loss 0.01585 variance 0.00076
Epoch: 6, loss: 0.00055
torch.Size([452, 1])
IC -0.04136 IC spear -0.07786 P_value 0.09827 loss 0.01295 variance 0.00076
Epoch: 7, loss: 0.00019
torch.Size([452, 1])
IC -0.04442 I

In [ ]:
taining(train_X,train_Y,test_X,test_Y,new_raw,0.7,5,150,0.001,3)

torch.Size([452, 1])
IC 0.09881 IC spear 0.0662 P_value 0.16001 loss 0.07629
Epoch: 0, loss: 0.32583
torch.Size([452, 1])
IC 0.09909 IC spear 0.06519 P_value 0.16646 loss 0.06386
Epoch: 1, loss: 0.29636
torch.Size([452, 1])
IC 0.09907 IC spear 0.06313 P_value 0.18032 loss 0.0538
Epoch: 2, loss: 0.27151
torch.Size([452, 1])
IC 0.09874 IC spear 0.06258 P_value 0.18418 loss 0.04563
Epoch: 3, loss: 0.25056
torch.Size([452, 1])
IC 0.09816 IC spear 0.05822 P_value 0.21668 loss 0.03893
Epoch: 4, loss: 0.23278
torch.Size([452, 1])
IC 0.09735 IC spear 0.05579 P_value 0.2365 loss 0.03334
Epoch: 5, loss: 0.21748
torch.Size([452, 1])
IC 0.09638 IC spear 0.05342 P_value 0.25706 loss 0.02858
Epoch: 6, loss: 0.20405
torch.Size([452, 1])
IC 0.09534 IC spear 0.05019 P_value 0.28695 loss 0.02444
Epoch: 7, loss: 0.19196
torch.Size([452, 1])
IC 0.09427 IC spear 0.04932 P_value 0.29539 loss 0.02077
Epoch: 8, loss: 0.18081
torch.Size([452, 1])
IC 0.09324 IC spear 0.04833 P_value 0.30526 loss 0.01748
Epoch: 

In [ ]:
features=150
seq_length = 5
ratio_=0.7
periods=5
interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_3m_US.csv',index_col=[0])
interest_rate=interest_rate.sort_values(by='Date')
interest_rate['range']=list(range(len(interest_rate)))
raw_data['change']=np.nan
for i in range(len(raw_data)):
  try:
    index=int(interest_rate[interest_rate['Date']<=int(raw_data.iloc[i,0])]['range'].tail(1))
    raw_data.iloc[i,-1]=(interest_rate.iloc[index+periods,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
  except:
    pass
new_raw=raw_data.dropna()
vectorizer_freq = CountVectorizer(max_features = features) 
total_vsm_freq = vectorizer_freq.fit_transform(new_raw.clean_review).toarray()
print("以词频为元素的文本-单词矩阵的维度是：\n\n",total_vsm_freq.shape)
from scipy import stats
vectorizer_tfidf=TfidfVectorizer(max_features=features)
total_vsm_tfidf=vectorizer_tfidf.fit_transform(new_raw.clean_review).toarray()
x, y = sliding_windows(total_vsm_tfidf,list(new_raw.change), seq_length)
train_size = int(len(x) * ratio_)
test_size = len(x) - train_size
train_Y=np.array(x[0:train_size])
from sklearn.preprocessing import RobustScaler
train_Y=np.array(y[0:train_size])
scaler =RobustScaler().fit(train_Y.reshape(-1, 1) )
train_Y = scaler.transform(train_Y.reshape(-1, 1)).reshape(1,-1)[0]
train_Y=Variable(torch.Tensor(train_Y))
train_X = Variable(torch.Tensor(np.array(x[0:train_size])))
test_X = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
test_Y = np.array(y[train_size:len(y)])

以词频为元素的文本-单词矩阵的维度是：

 (1512, 150)


In [ ]:
taining(train_X,train_Y,test_X,test_Y,new_raw,0.7,5,150,0.001,2)

torch.Size([452, 1])
IC -0.0171 IC spear 0.04219 P_value 0.37086 loss 0.13589
Epoch: 0, loss: 0.36359
torch.Size([452, 1])
IC -0.0138 IC spear 0.03984 P_value 0.39816 loss 0.10558
Epoch: 1, loss: 0.30872
torch.Size([452, 1])
IC -0.01105 IC spear 0.03866 P_value 0.41224 loss 0.08008
Epoch: 2, loss: 0.25993
torch.Size([452, 1])
IC -0.00879 IC spear 0.03543 P_value 0.45239 loss 0.05908
Epoch: 3, loss: 0.21695
torch.Size([452, 1])
IC -0.00693 IC spear 0.0334 P_value 0.47875 loss 0.04218
Epoch: 4, loss: 0.17941
torch.Size([452, 1])
IC -0.0054 IC spear 0.03337 P_value 0.47909 loss 0.02898
Epoch: 5, loss: 0.14684
torch.Size([452, 1])
IC -0.00412 IC spear 0.03204 P_value 0.49688 loss 0.0191
Epoch: 6, loss: 0.11879
torch.Size([452, 1])
IC -0.00303 IC spear 0.03013 P_value 0.52285 loss 0.01217
Epoch: 7, loss: 0.09479
torch.Size([452, 1])
IC -0.00209 IC spear 0.02883 P_value 0.5409 loss 0.00787
Epoch: 8, loss: 0.07444
torch.Size([452, 1])
IC -0.00127 IC spear 0.02779 P_value 0.55562 loss 0.00589
